In [ ]:
import pandas as pd
import numpy as np
import requests
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
import matplotlib.pyplot as plt
import seaborn as sns
import io
import xlsxwriter

# Download the MovieLens 100k dataset from a URL
movies_url = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.item'
ratings_url = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.data'

# Function to download data from a URL
def download_data(url):
    response = requests.get(url)
    response.raise_for_status()  # Ensure we got a successful response
    return io.StringIO(response.text)

# Load the MovieLens data
movies_file = download_data(movies_url)
ratings_file = download_data(ratings_url)

# Loading the movies and ratings data
movies = pd.read_csv(movies_file, sep='|', encoding='latin-1', header=None, names=['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url', 'unknown', 'action', 'adventure', 'animation', 'children', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'film_noir', 'horror', 'musical', 'mystery', 'romance', 'sci_fi', 'thriller', 'war', 'western'])
ratings = pd.read_csv(ratings_file, sep='\t', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Convert the timestamp to a readable format
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

# Create the Surprise dataset
reader = Reader(rating_scale=(1, 5))  # Ratings range from 1 to 5
data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)

# Train-Test Split (80% training, 20% testing)
trainset, testset = train_test_split(data, test_size=0.2)

# Initialize the SVD model
svd = SVD()

# Train the model
svd.fit(trainset)

# Evaluate the model on the testset
predictions = svd.test(testset)

# Calculate RMSE and MAE
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

# Define the function to get movie recommendations
def get_movie_recommendations(user_id, num_recommendations=5):
    all_movie_ids = movies['movie_id'].values
    predictions = [svd.predict(user_id, movie_id) for movie_id in all_movie_ids]
    predictions.sort(key=lambda x: x.est, reverse=True)
    top_recommendations = predictions[:num_recommendations]
    recommended_movie_ids = [pred.iid for pred in top_recommendations]
    recommended_movies = movies[movies['movie_id'].isin(recommended_movie_ids)]
    return recommended_movies[['movie_id', 'title']]

# Function to accept user ratings via CLI
def user_input_ratings():
    print("Welcome to the Movie Recommendation System!")
    ratings_input = []
    while True:
        try:
            user_id = int(input("Enter your User ID (an integer): "))
            movie_id = int(input("Enter Movie ID (an integer between 1 and 1682): "))
            rating = float(input("Enter your rating for this movie (between 1 and 5): "))
            ratings_input.append((user_id, movie_id, rating))
            more_input = input("Would you like to add another rating? (yes/no): ")
            if more_input.lower() != 'yes':
                break
        except ValueError:
            print("Invalid input, please enter valid values.")

    # Convert ratings input into a DataFrame
    ratings_df = pd.DataFrame(ratings_input, columns=['user_id', 'movie_id', 'rating'])
    return ratings_df

# Collect user ratings
user_ratings = user_input_ratings()

# Create a dataset from user input and train the model
user_data = Dataset.load_from_df(user_ratings[['user_id', 'movie_id', 'rating']], reader)
trainset = user_data.build_full_trainset()

# Train the SVD model
svd.fit(trainset)

# Get movie recommendations for the user
recommended_movies = get_movie_recommendations(user_id=user_ratings['user_id'].iloc[0], num_recommendations=5)
print("\nTop 5 Movie Recommendations for You:")
print(recommended_movies)

# Create an Excel file to store the results
with pd.ExcelWriter('Movie_Recommendation_Output.xlsx', engine='xlsxwriter') as writer:
    # Save recommended movies
    recommended_movies.to_excel(writer, sheet_name='Recommendations', index=False)

    # Save evaluation metrics (RMSE, MAE)
    metrics_df = pd.DataFrame({
        'Metric': ['RMSE', 'MAE'],
        'Value': [rmse, mae]
    })
    metrics_df.to_excel(writer, sheet_name='Metrics', index=False)

    # Save a histogram of ratings
    plt.figure(figsize=(8, 6))
    sns.histplot(ratings['rating'], bins=5, kde=False, color='blue')
    plt.title('Distribution of Ratings')
    plt.xlabel('Rating')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig('ratings_histogram.png')
    worksheet = writer.sheets['Recommendations']
    worksheet.insert_image('D2', 'ratings_histogram.png')

    # Save bar plot for RMSE and MAE
    metrics = [rmse, mae]
    metrics_labels = ['RMSE', 'MAE']
    plt.figure(figsize=(8, 6))
    sns.barplot(x=metrics_labels, y=metrics, palette='Blues_d')
    plt.title('Model Evaluation Metrics')
    plt.xlabel('Metric')
    plt.ylabel('Value')
    plt.tight_layout()
    plt.savefig('metrics_bar_plot.png')
    worksheet = writer.sheets['Metrics']
    worksheet.insert_image('D2', 'metrics_bar_plot.png')

print("Output saved to Movie_Recommendation_Output.xlsx")


RMSE: 0.9427
MAE:  0.7430
Welcome to the Movie Recommendation System!
